# Load Packages

In [54]:
# !pip install pydot
# !pip install keras-metrics

You are using pip version 18.0, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [63]:
# Keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding
from keras.models import model_from_json
from keras.preprocessing.text import text_to_word_sequence
from keras.utils.vis_utils import plot_model
from keras import metrics
import pydot

## Plot
import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)
import matplotlib as plt
# import matplotlib.pyplot

# # NLTK
# import nltk
# from nltk.corpus import stopwords
# nltk.download('stopwords')
# from nltk.stem import SnowballStemmer

# Other
import re
import string
import numpy as np
import pandas as pd
from sklearn.manifold import TSNE
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
# from sklearn import metrics
# from numpy import util


# Word2Vec
from gensim.test.utils import common_texts, get_tmpfile # not working
from gensim.models import Word2Vec

import re
from collections import Counter
import os


# Load orignal Dataset

In [ ]:
# original_csv= pd.read_csv('final_shuffled.csv')

In [ ]:
# original_csv

In [ ]:
# # tokenize. 

# original_tokenizer = Tokenizer()
# original_tokenizer.fit_on_texts(original_csv['8K_Content'])


# original_seq_len = 400 # set sequence length
# original_sequences = original_tokenizer.texts_to_sequences(original_csv['8K_Content'])
# original_data = pad_sequences(original_sequences, 
#                      maxlen=original_seq_len, padding='post', 
#                      truncating='post') # takes about 5-10 mins

In [ ]:
# original_csv['8K_Content_sequences'] = original_data.tolist()

In [ ]:
# original_csv.to_csv('')

In [ ]:
# original_csv.to_csv('final_shuffled_tokenized.csv')

In [ ]:
# original_csv['Dataset_Tag'] = original_csv[]

# Load downsampled

In [7]:
# downsampled_csv = pd.read_csv('final_downsampled.csv')

Loaded model from disk


# train test split

# Load Downsampled Dataset.

# Load Original Dataset

In [ ]:
# Load 

# Baseline Model 3,000 sequence
## not sure if i still have model json and h5, but we do have screenshot

# Baseline LSTM and LSTM CNN

In [ ]:
# 400 word sequence

In [14]:
# baseline_csv = downsampled_csv

In [22]:
# # tokenize words

# baseline_tokenizer = Tokenizer()
# baseline_tokenizer.fit_on_texts(baseline_csv['8K_Content'])


# baseline_seq_len = 400 # set sequence length


In [16]:
# baseline_sequences = baseline_tokenizer.texts_to_sequences(baseline_csv['8K_Content'])
# baseline_data = pad_sequences(baseline_sequences, 
#                      maxlen=baseline_seq_len, padding='post', 
#                      truncating='post') # takes about 5-10 mins

In [17]:
# baseline_csv['8K_Content_sequences'] = baseline_data.tolist()

In [18]:
# baseline_csv.to_csv('baseline.csv')

In [ ]:
# baseline_csv = pd.read_csv('baseline.csv')



In [19]:
# # 02-08 train. 09-10 develoment, 11-12 test
# # X_train = final_csv['8K_Content_cleaned'][final_csv['Year'] <= 2008]
# # X_dev = final_csv['8K_Content_cleaned'][(final_csv['Year'] >= 2009) & 
# #                                        (final_csv['Year'] <= 2010)]
# # X_test = final_csv['8K_Content_cleaned'][final_csv['Year'] >= 2011]

# X_train = baseline_csv['8K_Content_sequences'][baseline_csv['Year'] <= 2008]
# X_dev = baseline_csv['8K_Content_sequences'][(baseline_csv['Year'] >= 2009) & 
#                                        (baseline_csv['Year'] <= 2010)]
# X_test = baseline_csv['8K_Content_sequences'][baseline_csv['Year'] >= 2011]

# y_train = baseline_csv[['down','stay','up']][baseline_csv['Year'] <= 2008]
# y_dev = baseline_csv[['down','stay','up']][(baseline_csv['Year'] >= 2009) & 
#                                        (baseline_csv['Year'] <= 2010)]
# y_test = baseline_csv[['down','stay','up']][baseline_csv['Year'] >= 2011]

In [21]:
# # precision, f1, recall
# # load model
# # note: latest file updated 11/30. 3,000 word sequence.

# # load json and create model
# json_file = open('model_lstm_1.json', 'r')
# baseline_model_json = json_file.read()
# json_file.close()
# baseline_model = model_from_json(baseline_model_json)

# # load weights into new model
# baseline_model.load_weights("model_lstm_1.h5")
# print("Loaded model from disk")




Loaded model from disk


In [49]:
# # evaluate loaded model on test data
# baseline_model.compile(loss='categorical_crossentropy', optimizer='adam', 
#                        metrics=['accuracy'])

# # loaded_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# baseline_score = baseline_model.evaluate(np.array(X_test.values.tolist()), 
#                               np.array(y_test.values.tolist()), verbose=0)
# print("%s: %.2f%%" % (baseline_model.metrics_names[1], baseline_score[1]*100))

acc: 35.14%


In [ ]:
# accuracy is low because used different tokenizer method

In [25]:
# baseline_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 400, 100)          92619500  
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 303       
Total params: 92,700,203
Trainable params: 92,700,203
Non-trainable params: 0
_________________________________________________________________


In [35]:
# plot_model(baseline_model, 
#            to_file='baseline_model_plot.png', show_shapes=True, show_layer_names=True)

In [ ]:
# # predictions

# predict(X_test, y_test)

In [ ]:
# dropout visuals

# LSTM and LSTM CNN: Downsampled

# Model_Downsampled_IncreaseSeqLen.ipynb

# Model_Downsampled_DecreaseSeqLen.ipynb

In [51]:
# downsampled_tokenizer = Tokenizer()
# downsampled_tokenizer.fit_on_texts(downsampled_csv['8K_Content'])


# downsampled_seq_len = 300 # set sequence length
# downsampled_sequences = downsampled_tokenizer.texts_to_sequences(downsampled_csv['8K_Content'])
# downsampled_data = pad_sequences(downsampled_sequences, 
#                      maxlen=downsampled_seq_len, padding='post', 
#                      truncating='post') # takes about 5-10 mins

In [52]:
# downsampled_csv['8K_Content_sequences'] = downsampled_data.tolist()

In [ ]:
downsampled_csv = pd.read_csv('downsampled_tokenized.csv')

In [53]:
# run this for downsampled

X_train = downsampled_csv['8K_Content_sequences'][downsampled_csv['Year'] <= 2008]
X_dev = downsampled_csv['8K_Content_sequences'][(downsampled_csv['Year'] >= 2009) & 
                                       (downsampled_csv['Year'] <= 2010)]
X_test = downsampled_csv['8K_Content_sequences'][downsampled_csv['Year'] >= 2011]

y_train = downsampled_csv[['down','stay','up']][downsampled_csv['Year'] <= 2008]
y_dev = downsampled_csv[['down','stay','up']][(downsampled_csv['Year'] >= 2009) & 
                                       (downsampled_csv['Year'] <= 2010)]
y_test = downsampled_csv[['down','stay','up']][downsampled_csv['Year'] >= 2011]

In [57]:
# downsampled_csv.to_csv('downsampled_tokenized.csv')

In [58]:
# load model

model_num = 4

# load json and create model
json_file = open('model_lstm_%d.json' %model_num, 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

# load weights into new model
loaded_model.load_weights("model_lstm_%d.h5" %model_num)
print("Loaded model from disk: ", 'model_lstm_%d.json and model_lstm_%d.h5' %(model_num, model_num))

Loaded model from disk:  model_lstm_4.json and model_lstm_4.h5


In [65]:
# evaluate loaded model on test data
loaded_model.compile(loss='categorical_crossentropy', optimizer='adam', 
                     metrics=[metrics.categorical_accuracy])
# loaded_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
score = loaded_model.evaluate(np.array(X_test.values.tolist()), 
                              np.array(y_test.values.tolist()), verbose=0)
print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))

categorical_accuracy: 44.36%


# Model_Downsampled_DecreaseSeqLen_IncreaseEpochs.ipynb

# Model_Downsampled_DecreaseSeqLen_IncreaseEpochs_IncreaseDropout.ipynb